In [1]:
!pip install -r requirements.txt

     |████████████████████████████████| 1.4 MB 21.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.8 MB 107.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 86.0 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 6.0 MB/s  eta 0:00:01
     |████████████████▍               | 411.7 MB 111.7 MB/s eta 0:00:04   |▊                               | 16.9 MB 49.3 MB/s eta 0:00:16     |███████████▎                    | 283.4 MB 117.4 MB/s eta 0:00:05

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |███████████████████████████▎    | 684.3 MB 83.9 MB/s eta 0:00:023     |████████████████████            | 503.1 MB 111.7 MB/s eta 0:00:03

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 804.1 MB 5.1 kB/s s eta 0:00:01
     |████████████████████████████████| 64 kB 143 kB/s  eta 0:00:01
     |████████████████████████████████| 97 kB 678 kB/s  eta 0:00:01
     |████████████████████████████████| 131 kB 97.2 MB/s eta 0:00:01
  Using cached GitPython-3.1.17-py3-none-any.whl (166 kB)
  Using cached gitdb-4.0.7-py3-none-any.whl (63 kB)
  Using cached smmap-4.0.0-py2.py3-none-any.whl (24 kB)
     |████████████████████████████████| 21.6 MB 44.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.9 MB 66.5 MB/s eta 0:00:01
     |████████████████████████████████| 806 kB 26.8 MB/s eta 0:00:01
     |████████████████████████████████| 111 kB 100.4 MB/s eta 0:00:01
     |████████████████████████████████| 10.6 MB 53.3 MB/s eta 0:00:01
     |████████████████████████████████| 274 kB 111.6 MB/s eta 0:00:01
     |████████████████████████████████| 129 kB 77.7 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 64.1 MB/s eta 0:00:01
 

In [7]:
!aws s3 cp s3://s3illustration/metadata . --recursive

download: s3://s3illustration/metadata/vqgan.1024.config.yml to ./vqgan.1024.config.yml
download: s3://s3illustration/metadata/Validation_GCC-1.1.0-Validation.tsv to ./Validation_GCC-1.1.0-Validation.tsv
download: s3://s3illustration/metadata/Train_GCC-dalle.tsv to ./Train_GCC-dalle.tsv
download: s3://s3illustration/metadata/Train_GCC-training.tsv to ./Train_GCC-training.tsv
download: s3://s3illustration/metadata/vqgan.1024.model.ckpt to ./vqgan.1024.model.ckpt


In [4]:
!mv ./vqgan.1024.config.yml ~/.cache/vqgan.1024.config.yml

In [5]:
!mv ./vqgan.1024.model.ckpt ~/.cache/vqgan.1024.model.ckpt

### Create dalle training dataset

In [6]:
import pandas as pd
import zlib

df = pd.read_csv('Train_GCC-training.tsv', sep='\t', header=None)
df.columns=['caption','url']

# for i, row in df.iterrows():
#     unique = (zlib.crc32(row['caption'].encode('utf-8')) & 0xffffffff)
#     fname = f'{i}_{unique}'

In [18]:
df['index'] = df.index

In [20]:
%%time

df['filename'] = df.apply(
    lambda row: f"{row['index']}_{(zlib.crc32(row['url'].encode('utf-8')) & 0xffffffff)}",
    axis=1
)

CPU times: user 36.7 s, sys: 668 ms, total: 37.4 s
Wall time: 37.4 s


In [22]:
df[['caption', 'filename']].to_csv('Train_GCC-dalle.tsv', sep='\t', index=False)

In [25]:
!aws s3 cp Train_GCC-dalle.tsv s3://s3illustration/metadata/Train_GCC-dalle.tsv

upload: ./Train_GCC-dalle.tsv to s3://s3illustration/metadata/Train_GCC-dalle.tsv


In [ ]:
!python3 ../train_dalle.py --dalle_path ./dalle.pt --image_text_folder ../adamonkey --taming # have to run it first in terminal for wandb creds

Working with z of shape (1, 256, 16, 16) = 65536 dimensions.
loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips/vgg.pth
VQLPIPSWithDiscriminator running with hinge loss.
Loading training data finished.
Creating text_files finished.
3318333 image-text pairs found for training
wandb: Currently logged in as: adamonkey (use `wandb login --relogin` to force relogin)
wandb: Tracking run with wandb version 0.10.31
wandb: Resuming run clear-smoke-13
wandb: ⭐️ View project at https://wandb.ai/adamonkey/dalle_train_transformer
wandb: 🚀 View run at https://wandb.ai/adamonkey/dalle_train_transformer/runs/70ddzrm4
wandb: Run data is saved locally in /home/ubuntu/env/DALLE-pytorch/adamonkey/wandb/run-20210603_105043-70ddzrm4
wandb: Run `wandb offline` to turn off syncing.

0 0 loss - 7.405618667602539
0 9 sample_per_sec - 2.458562500808727
0 10 loss - 7.143937587738037
0 19 sample_per_sec - 4.6324272747687605
0 20 loss - 6.694444179534912
0 29 sample_per_sec - 4.496362675154749
An ex